In [1]:
import pandas as pd

In [65]:
c_met = pd.read_csv("../HA_data/metadata/crystals_metadata.csv", index_col=0)
c_seq = pd.read_csv("../HA_data/aligned_sequences/crystals_aln.csv", index_col=0)

# NOT the same
# I needed to add header: id,chain,host,location,year
m_met = pd.read_csv("../HA_data/metadata/models_metadata.csv", index_col=0, na_values=["-"])
m_seq = pd.read_csv("../HA_data/aligned_sequences/models_aln.csv", index_col=0)
m_template = pd.read_csv("../HA_data/models_template_mapping.csv", index_col=0)

# Crystal data

In [8]:
c_met.head()

,chain,subtype,host,location,year
id,,,,,
1ha0,A,H3N2,NaN,Aichi,1968
4eef,A,H1N1,NaN,BrevigMission,1918
4cqp,A,H5N1,NaN,Vietnam,2004
4cqq,A,H5N1,NaN,Vietnam,2004
4cqr,A,H5N1,NaN,Vietnam,2004


In [18]:
"H3N2".split("\w")

['H3N2']

In [20]:
len(c_met.location.value_counts())

48

In [21]:
c_met.host.value_counts()

avian                     50
canine                     7
equine                     7
reassortant X31            6
swine                      5
harbor_seal                2
bat                        2
reassortant IDCDC_RG29     1
dtype: int64

In [25]:
def clean_host(x):
    if pd.isnull(x):
        return "human"
    elif x.find("reassortant") > -1:
        return "reassortant"
    else:
        return x.replace("_", " ")

In [26]:
c_met.host = c_met.host.apply(clean_host)

In [31]:
c_met["H"] = c_met.subtype.apply(lambda x: int(x.split("N")[0][1:]))
c_met["N"] = c_met.subtype.apply(lambda x: int(x.split("N")[1]))

In [32]:
c_seq.head()

,sequence
id,
1eo8,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1ha0,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hgf,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hgg,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hgh,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...


In [33]:
# sequence for other data missing (but not the other way)
c_seq.index.difference(c_met.index)

Index(['2rft', '2rfu', '2wr0', '2wr1', '2wr2', '2wr3', '2wr4', '2wr5', '2yp2',
       '2yp3', '2yp4', '2yp5', '3bt6', '4fqj', '4fqk', '4fqm', '4kon', '4lcx',
       '4lkg', '4lkh', '4lki', '4lkj', '4lkk', '4m40', '4m44', '4mc5', '4mhh',
       '4nrj', '4nrk', '4nrl', '4qy0', '4qy1', '4qy2'],
      dtype='object', name='id')

In [35]:
c_all = c_met.join(c_seq, how="outer")

In [76]:
c_all.head()

,chain,subtype,host,location,year,H,N,sequence
id,,,,,,,,
1eo8,A,H3N2,human,Aichi,1968,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1ha0,A,H3N2,human,Aichi,1968,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hgd,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hge,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...
1hgf,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICN-N...


In [38]:
c_all.to_csv("../site/data/crystals_metadata.csv", float_format = '%.0f')  # for missing H/N

## Model data

In [66]:
m_met.head()

,chain,host,location,year
p_id,,,,
AIB06038,A,swine,Nebraska,2014
ABW36333,A,swine,Ontario,1981
ADD23162,A,NaN,San Diego,2009
AAV80798,A,avian,Minnesota,2003
ABY40410,A,swine,Saraburi,2005


In [67]:
m_met.host.value_counts()

swine             482
avian             316
emu                 2
ferret              1
rhea                1
giant anteater      1
dtype: int64

In [68]:
m_met.host = m_met.host.fillna("human")

In [69]:
len(m_met.location.value_counts())

323

In [70]:
m_seq.head()

,sequence
id,
AAA43090,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43247,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO38701,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO44057,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO52247,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...


In [71]:
m_template.head()

,template_id
p_id,
AAA43578,2wr5
AEK49568,4wsw
AGL58994,4wsw
AGL60617,4wsw
AGL60775,4wsw


In [74]:
m_all = m_met.join(m_template, how="outer")
m_all = m_all.join(m_seq, how="outer")
m_all.index.name = "p_id"

In [75]:
m_all.head()

,chain,host,location,year,template_id,sequence
p_id,,,,,,
AAA43090,A,human,Berlin,1964,3ku6,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43117,A,human,duck,1972,3ku6,GDQICIGYHSN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43185,A,human,Japan,1957,3ku6,GDQICIGYHAN-STEKVDTNLERNVTVTHAKDILEK--THNGKLCK...
AAA43205,A,avian,Ontario,1966,1jso,-DQICIGYHAN-STKQVDTIMEKNVTVTYAQDILEK--EHNGKLCS...
AAA43243,A,avian,Hong Kong,1978,3ku6,GDQICIGYHAN-STETVDTILERNVTVTHAKNILEK--THNGKLCK...


In [77]:
# WARNING: as of now without serotype or subtype
m_all.to_csv("../site/data/models_metadata.csv", float_format = '%.0f')